In [ ]:
import os 
import numpy as np
from catboost import Pool,CatBoostRegressor
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,accuracy_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV


# Base path
DATA_PATH = "raw_data"



In [2]:
%pwd

'/home/leksman/Desktop/my git hub work/end_to_end_Real_Estate_Demand_Predictio/notebook'

# -------------------------------
# Step 1: Load raw CSV files
# Step 2: Merge into a single dataset
# Step 3: Preprocess (fill missing values, add prefixes, optimize ints)
# -------------------------------


In [3]:

# -------------------------------
# Setup
# -------------------------------
os.chdir("../")  # optional, change if needed
DATA_PATH = "raw_data"

# -------------------------------
# Helper: add prefixes to columns
# -------------------------------
def prefix_columns(df: pd.DataFrame, prefix: str) -> pd.DataFrame:
    """Add a prefix to all columns except sector and month."""
    rename_map = {col: f"{prefix}{col}" for col in df.columns if col not in ["sector", "month"]}
    return df.rename(columns=rename_map)

# -------------------------------
# Train Data
# -------------------------------
ci = (
    pd.read_csv(f"{DATA_PATH}/train/city_indexes.csv")
    .head(6)
    .fillna(-1)
    .drop(columns=["total_fixed_asset_investment_10k"])
    .pipe(prefix_columns, "ci_")
)

csi = pd.read_csv(f"{DATA_PATH}/train/city_search_index.csv")

sp = pd.read_csv(f"{DATA_PATH}/train/sector_POI.csv").fillna(-1).pipe(prefix_columns, "sp_")
train_lt = pd.read_csv(f"{DATA_PATH}/train/land_transactions.csv").pipe(prefix_columns, "lt_")
train_ltns = pd.read_csv(f"{DATA_PATH}/train/land_transactions_nearby_sectors.csv").pipe(prefix_columns, "ltns_")
train_pht = pd.read_csv(f"{DATA_PATH}/train/pre_owned_house_transactions.csv").pipe(prefix_columns, "pht_")
train_phtns = pd.read_csv(f"{DATA_PATH}/train/pre_owned_house_transactions_nearby_sectors.csv").pipe(prefix_columns, "phtns_")
train_nht = pd.read_csv(f"{DATA_PATH}/train/new_house_transactions.csv").pipe(prefix_columns, "nht_")
train_nhtns = pd.read_csv(f"{DATA_PATH}/train/new_house_transactions_nearby_sectors.csv").pipe(prefix_columns, "nhtns_")

# -------------------------------
# Test Data
# -------------------------------
test = pd.read_csv(f"{DATA_PATH}/test.csv")
test[["month", "sector"]] = test["id"].str.split("_", expand=True)

# -------------------------------
# Month mapping
# -------------------------------
MONTH_CODES = {
    "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4,
    "May": 5, "Jun": 6, "Jul": 7, "Aug": 8,
    "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12
}

# -------------------------------
# Step 1: Base dataset
# -------------------------------
sectors = pd.DataFrame({"sector": train_nht["sector"].unique().tolist() + ["sector 95"]})
months = pd.DataFrame({"month": train_nht["month"].unique()})

data = months.merge(sectors, how="cross")
data["sector_id"] = data["sector"].str.split(" ").str[1].astype("int16")
data["year"] = data["month"].str.split("-").str[0].astype("int16")
data["month_num"] = data["month"].str.split("-").str[1].map(MONTH_CODES).astype("int8")
data["time"] = ((data["year"] - 2019) * 12 + data["month_num"] - 1).astype("int16")
data = data.sort_values(["sector_id", "time"])

# -------------------------------
# Step 2: Join features
# -------------------------------
data = (
    data.merge(train_nht, on=["sector", "month"], how="left").fillna(0)
        .merge(train_nhtns, on=["sector", "month"], how="left").fillna(-1)
        .merge(train_pht, on=["sector", "month"], how="left").fillna(-1)
        .merge(train_phtns, on=["sector", "month"], how="left").fillna(-1)
        .merge(ci.rename(columns={"ci_city_indicator_data_year": "year"}), on="year", how="left").fillna(-1)
        .merge(sp, on="sector", how="left").fillna(-1)
        .merge(train_lt, on=["sector", "month"], how="left").fillna(-1)
        .merge(train_ltns, on=["sector", "month"], how="left").fillna(-1)
)

# -------------------------------
# Step 3: Optimize ints
# -------------------------------
for col in data.select_dtypes(include=["int64"]).columns:
    c_min, c_max = data[col].min(), data[col].max()
    if c_min == 0 and c_max == 0:
        data.drop(columns=[col], inplace=True)
    elif np.iinfo(np.int8).min <= c_min <= np.iinfo(np.int8).max:
        data[col] = data[col].astype("int8")
    elif np.iinfo(np.int16).min <= c_min <= np.iinfo(np.int16).max:
        data[col] = data[col].astype("int16")
    elif np.iinfo(np.int32).min <= c_min <= np.iinfo(np.int32).max:
        data[col] = data[col].astype("int32")

data = data.drop(columns=["month", "sector", "year"])

# -------------------------------
# Step 4: Rolling features
# -------------------------------
data = data.sort_values(["sector_id", "time"])
for col in data.columns[3:]:
    for p in [3, 6, 12]:
        data[f"{col}_mean{p}"] = data.groupby("sector_id")[col].transform(lambda x: x.rolling(p, min_periods=1).mean())
        data[f"{col}_min{p}"] = data.groupby("sector_id")[col].transform(lambda x: x.rolling(p, min_periods=1).min())
        data[f"{col}_max{p}"] = data.groupby("sector_id")[col].transform(lambda x: x.rolling(p, min_periods=1).max())

# -------------------------------
# Step 5: Lag + Cyclical features
# -------------------------------
lag = 1
data["label"] = data.groupby("sector_id")["nht_amount_new_house_transactions"].shift(lag)

# cyclical encoding
data["cs"] = np.cos((data["month_num"] - 1) / 6 * np.pi)
data["sn"] = np.sin((data["month_num"] - 1) / 6 * np.pi)
data["cs6"] = np.cos((data["month_num"] - 1) / 3 * np.pi)
data["sn6"] = np.sin((data["month_num"] - 1) / 3 * np.pi)
data["cs3"] = np.cos((data["month_num"] - 1) / 1.5 * np.pi)
data["sn3"] = np.sin((data["month_num"] - 1) / 1.5 * np.pi)

data = data.drop(columns=["sector_id"])

# -------------------------------
# Step 6: Train/Test Split
# -------------------------------
cat_features = ["month_num"]

N_TEST_MONTHS = 3
max_time = data["time"].max()
border = max_time - N_TEST_MONTHS

train_df = data[data["time"] <= border].dropna(subset=["label"])
test_df = data[data["time"] > border].dropna(subset=["label"])

trainPool = Pool(
    data=train_df.drop(columns=["label"]).fillna(-2),
    label=train_df["label"],
    cat_features=cat_features,
)

testPool = Pool(
    data=test_df.drop(columns=["label"]).fillna(-2),
    label=test_df["label"],
    cat_features=cat_features,
)

print("✅ Train/Test Pools ready!")
print("Train size:", train_df.shape)
print("Test size:", test_df.shape)


/tmp/ipykernel_273933/1034085068.py:101: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f"{col}_min{p}"] = data.groupby("sector_id")[col].transform(lambda x: x.rolling(p, min_periods=1).min())
/tmp/ipykernel_273933/1034085068.py:102: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f"{col}_max{p}"] = data.groupby("sector_id")[col].transform(lambda x: x.rolling(p, min_periods=1).max())
/tmp/ipykernel_273933/1034085068.py:100: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

✅ Train/Test Pools ready!
Train size: (6048, 2479)
Test size: (288, 2479)


In [4]:
train_df.describe()

,month_num,time,nht_num_new_house_transactions,nht_area_new_house_transactions,nht_price_new_house_transactions,nht_amount_new_house_transactions,nht_area_per_unit_new_house_transactions,nht_total_price_per_unit_new_house_transactions,nht_num_new_house_available_for_sale,nht_area_new_house_available_for_sale,...,ltns_transaction_amount_nearby_sectors_mean12,ltns_transaction_amount_nearby_sectors_min12,ltns_transaction_amount_nearby_sectors_max12,label,cs,sn,cs6,sn6,cs3,sn3
count,6048.000000,6048.000000,6048.000000,6048.000000,6048.000000,6048.000000,6048.000000,6048.000000,6048.000000,6.048000e+03,...,6048.000000,6048.000000,6048.000000,6048.000000,6.048000e+03,6.048000e+03,6048.000000,6.048000e+03,6.048000e+03,6.048000e+03
mean,6.333333,32.000000,76.221892,8233.837136,37125.925761,27674.978877,108.778274,522.767811,916.534226,1.061649e+05,...,12504.831276,-0.218750,75002.894164,27586.841475,2.168294e-02,3.755596e-02,-0.015873,2.749287e-02,-2.784369e-16,7.209226e-17
std,3.455211,18.185746,154.780902,15982.448830,29066.336594,47084.460640,72.966695,619.300369,1503.110613,1.601766e+05,...,14881.224194,0.413433,84752.286920,47031.837253,7.011951e-01,7.117653e-01,0.706987,7.066305e-01,7.071652e-01,7.071652e-01
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,-1.000000,-1.000000,-1.000000,0.000000,-1.000000e+00,-1.000000e+00,-1.000000,-8.660254e-01,-5.000000e-01,-8.660254e-01
25%,3.000000,16.000000,3.000000,441.000000,15793.750000,1559.390000,92.000000,173.532500,67.000000,9.839250e+03,...,0.000000,0.000000,0.000000,1530.742500,-5.000000e-01,-5.000000e-01,-0.500000,-8.660254e-01,-5.000000e-01,-8.660254e-01
50%,6.000000,32.000000,23.000000,2899.000000,32043.000000,11254.130000,107.000000,361.305000,381.000000,5.113650e+04,...,7788.834722,0.000000,53296.000000,11210.960000,6.123234e-17,1.224647e-16,-0.500000,1.224647e-16,-5.000000e-01,-2.449294e-16
75%,9.000000,48.000000,79.000000,8889.250000,52315.250000,32783.045000,128.000000,619.425000,1257.000000,1.536355e+05,...,19460.340530,0.000000,113015.666700,32613.722500,8.660254e-01,8.660254e-01,0.500000,8.660254e-01,1.000000e+00,8.660254e-01
max,12.000000,63.000000,2669.000000,294430.000000,208288.000000,606407.640000,2003.000000,7803.600000,12048.000000,1.220617e+06,...,102270.183333,0.000000,504823.200000,606407.640000,1.000000e+00,1.000000e+00,1.000000,8.660254e-01,1.000000e+00,8.660254e-01


In [5]:
test_df

,month_num,time,nht_num_new_house_transactions,nht_area_new_house_transactions,nht_price_new_house_transactions,nht_amount_new_house_transactions,nht_area_per_unit_new_house_transactions,nht_total_price_per_unit_new_house_transactions,nht_num_new_house_available_for_sale,nht_area_new_house_available_for_sale,...,ltns_transaction_amount_nearby_sectors_mean12,ltns_transaction_amount_nearby_sectors_min12,ltns_transaction_amount_nearby_sectors_max12,label,cs,sn,cs6,sn6,cs3,sn3
64,5,64,5.0,524.0,80692.0,4229.40,105.0,845.88,476.0,47767.0,...,47012.332812,0.0,166591.00000,3622.08,-0.500000,8.660254e-01,-0.5,-8.660254e-01,-0.5,8.660254e-01
65,6,65,26.0,2570.0,75661.0,19445.53,99.0,747.90,452.0,45502.0,...,47012.332812,0.0,166591.00000,4229.40,-0.866025,5.000000e-01,0.5,-8.660254e-01,-0.5,-8.660254e-01
66,7,66,13.0,1267.0,73385.0,9295.32,97.0,715.02,445.0,44872.0,...,42319.864062,0.0,166591.00000,19445.53,-1.000000,1.224647e-16,1.0,-2.449294e-16,1.0,-4.898587e-16
131,5,64,23.0,2824.0,13575.0,3833.23,123.0,166.66,964.0,115482.0,...,0.000000,0.0,0.00000,3155.24,-0.500000,8.660254e-01,-0.5,-8.660254e-01,-0.5,8.660254e-01
132,6,65,149.0,13445.0,12283.0,16514.18,90.0,110.83,970.0,115797.0,...,0.000000,0.0,0.00000,3833.23,-0.866025,5.000000e-01,0.5,-8.660254e-01,-0.5,-8.660254e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6363,6,65,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,-1.000000,-1.0,-1.00000,0.00,-0.866025,5.000000e-01,0.5,-8.660254e-01,-0.5,-8.660254e-01
6364,7,66,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,-1.000000,-1.0,-1.00000,0.00,-1.000000,1.224647e-16,1.0,-2.449294e-16,1.0,-4.898587e-16
6429,5,64,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,15124.190477,0.0,85408.85714,0.00,-0.500000,8.660254e-01,-0.5,-8.660254e-01,-0.5,8.660254e-01
6430,6,65,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,12522.380953,0.0,85408.85714,0.00,-0.866025,5.000000e-01,0.5,-8.660254e-01,-0.5,-8.660254e-01


In [6]:
train_df = train_df[train_df["label"] != 0]
test_df = test_df[test_df["label"] != 0]

In [7]:
train_df

,month_num,time,nht_num_new_house_transactions,nht_area_new_house_transactions,nht_price_new_house_transactions,nht_amount_new_house_transactions,nht_area_per_unit_new_house_transactions,nht_total_price_per_unit_new_house_transactions,nht_num_new_house_available_for_sale,nht_area_new_house_available_for_sale,...,ltns_transaction_amount_nearby_sectors_mean12,ltns_transaction_amount_nearby_sectors_min12,ltns_transaction_amount_nearby_sectors_max12,label,cs,sn,cs6,sn6,cs3,sn3
1,2,1,24.0,2526.0,34846.0,8802.81,105.0,366.78,158.0,15814.0,...,23167.250000,0.0,46334.50000,13827.14,8.660254e-01,0.500000,0.5,8.660254e-01,-0.5,8.660254e-01
2,3,2,68.0,6732.0,34589.0,23283.48,99.0,342.40,151.0,14767.0,...,15444.833333,0.0,46334.50000,8802.81,5.000000e-01,0.866025,-0.5,8.660254e-01,-0.5,-8.660254e-01
3,4,3,69.0,6935.0,38392.0,26626.68,101.0,385.89,141.0,12936.0,...,11583.625000,0.0,46334.50000,23283.48,6.123234e-17,1.000000,-1.0,1.224647e-16,1.0,-2.449294e-16
4,5,4,47.0,3829.0,22587.0,8649.42,81.0,184.03,141.0,12936.0,...,9266.900000,0.0,46334.50000,26626.68,-5.000000e-01,0.866025,-0.5,-8.660254e-01,-0.5,8.660254e-01
5,6,5,41.0,4260.0,47216.0,20116.16,104.0,490.64,158.0,16788.0,...,7722.416667,0.0,46334.50000,8649.42,-8.660254e-01,0.500000,0.5,-8.660254e-01,-0.5,-8.660254e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6384,8,19,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,12733.845237,0.0,75230.85714,463.55,-8.660254e-01,-0.500000,0.5,8.660254e-01,-0.5,8.660254e-01
6386,10,21,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,18905.726190,0.0,75230.85714,914.99,-1.836970e-16,-1.000000,-1.0,3.673940e-16,1.0,-7.347881e-16
6389,1,24,2.0,352.0,52694.0,1853.04,176.0,926.52,6.0,1277.0,...,12636.488095,0.0,66996.85714,1274.87,1.000000e+00,0.000000,1.0,0.000000e+00,1.0,0.000000e+00
6390,2,25,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,12636.488095,0.0,66996.85714,1853.04,8.660254e-01,0.500000,0.5,8.660254e-01,-0.5,8.660254e-01


In [8]:
any(train_df.isnull().sum())

False

In [9]:
X_train = train_df.drop("label",axis=1)

In [10]:
y_train = train_df["label"]

In [11]:
# Identify feature types
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()



preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
    ]
)

In [12]:
# -------------------------------
models_and_parameters = {
    "RandomForest": (
        RandomForestRegressor(random_state=42),
        {"model__n_estimators": [100, 200],
         "model__max_depth": [5, 10, None]}
    ),
    "XGBoost": (
        XGBRegressor(random_state=42, verbosity=0),
        {"model__n_estimators": [200, 500],
         "model__max_depth": [4, 6, 8]}
    ),
    "LightGBM": (
        LGBMRegressor(random_state=42),
        {"model__n_estimators": [200, 500],
         "model__num_leaves": [31, 64]}
    ),
    "CatBoost": (
        CatBoostRegressor(verbose=0, random_state=42),
        {"model__iterations": [500, 1000],
         "model__depth": [6, 8]}
    )
}


In [ ]:
results = []

for name, (model, params) in models_and_parameters.items():
    print(f"\n🔍 Running GridSearch for {name}...")
    
    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("model", model)
    ])
    
    grid = RandomizedSearchCV(
    pipeline,
    param_distributions=params,   # same dict
    n_iter=5,                     # try only 5 random combinations
    cv=3,
    scoring="neg_mean_squared_error",
    n_jobs=-1,
    random_state=42
)
    
    grid.fit(X_train, y_train)
    
    best_model = grid.best_estimator_
    best_score = -grid.best_score_  # convert to positive MSE
    
    print(f"✅ {name} best params: {grid.best_params_}")
    print(f"✅ {name} CV MSE: {best_score:.4f}")
    
    results.append({
        "Model": name,
        "Best Params": grid.best_params_,
        "CV MSE": best_score
    })

# -------------------------------
# Step 4: Compare models
# -------------------------------
results_df = pd.DataFrame(results)
print("\n📊 Model Comparison:")
print(results_df)


🔍 Running GridSearch for RandomForest...
✅ RandomForest best params: {'model__n_estimators': 200, 'model__max_depth': None}
✅ RandomForest CV MSE: 406679006.0713

🔍 Running GridSearch for XGBoost...


In [ ]:
y_tran_pred =  best_model.predict(X_train)

In [ ]:
np.sqrt(r2_score(y_true=y_train,y_pred=y_tran_pred))

np.float64(0.984735235389213)